In [1]:
print("hej")

hej


In [4]:
'''This script goes along the blog post
"Building powerful image classification models using very little data"
from blog.keras.io.
It uses data that can be downloaded at:
https://www.kaggle.com/c/dogs-vs-cats/data
In our setup, we:
- created a data/ folder
- created train/ and validation/ subfolders inside data/
- created cats/ and dogs/ subfolders inside train/ and validation/
- put the cat pictures index 0-999 in data/train/cats
- put the cat pictures index 1000-1400 in data/validation/cats
- put the dogs pictures index 12500-13499 in data/train/dogs
- put the dog pictures index 13500-13900 in data/validation/dogs
So that we have 1000 training examples for each class, and 400 validation examples for each class.
In summary, this is our directory structure:
```
data/
    train/
        dogs/
            dog001.jpg
            dog002.jpg
            ...
        cats/
            cat001.jpg
            cat002.jpg
            ...
    validation/
        dogs/
            dog001.jpg
            dog002.jpg
            ...
        cats/
            cat001.jpg
            cat002.jpg
            ...
```
'''

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K


# dimensions of our images.
img_width, img_height = 150, 150

# flowdhub folder
flowdhub = 'https://www.floydhub.com/viewer/data/xLHRt9d9UnipGEEr4FkWdF/'
flowhubtrain = 'https://www.floydhub.com/viewer/data/xLHRt9d9UnipGEEr4FkWdF/train'
flowdhubvalidation = 'https://www.floydhub.com/viewer/data/xLHRt9d9UnipGEEr4FkWdF/validation'

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 2
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

model.save_weights('first_try.h5')
saved_model = model.to_json()
with open('first_try.json','w') as f:
    f.write(saved_model)

Found 2002 images belonging to 2 classes.
Found 801 images belonging to 2 classes.
Epoch 1/2
125/125 [==============================] - 57s - loss: 0.7260 - acc: 0.5070 - val_loss: 0.6839 - val_acc: 0.6038

In [5]:
model.save_weights('first_try.h5')
saved_model = model.to_json()
with open('first_try.json','w') as f:
    f.write(saved_model)

In [9]:
from keras.models import model_from_json
# Load trained model
# load json and create model
json_file = open('first_try.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
global loaded_model
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("first_try.h5")
print("Loaded model from disk")

Loaded model from disk


In [40]:
import scipy.misc

testimage = scipy.misc.imresize(scipy.misc.imread('data/validation/dogs/dog.1001.jpg'), (150, 150))
testimage = testimage.reshape((1,) + testimage.shape)

In [41]:
testimage.shape
prediction = loaded_model.predict(testimage)
#print(loaded_model.predict(np.swapaxes(np.array(data), 0, 1)))
print(prediction)

if (prediction[0] == 1):
        print("Cat")
        one = True
else:
    print("Dog")


[[ 0.05394386]]
Dog


In [14]:
testimage.shape

(150, 150, 3)

In [31]:
import numpy as np
testimage.reshape((1,) + testimage.shape).shape

(1, 150, 150, 3)

In [28]:
(testimage.shape,1)

((150, 150, 3), 1)